# 1. The taxi driving problem (Taksi kullanma sorunu) 🚕 

Bir ajana Pekiştirmeli Öğrenme (Reinforcement Learning) kullanarak taksi sürmeyi öğreteceğiz.

Gerçek dünyada taksi sürmek, başlamak için çok karmaşık bir iştir. Bu nedenle, iyi bir taksi şoförünün yaptığı 3 temel şeyi içeren basitleştirilmiş bir ortamda çalışacağız, bunlar:

- yolcuları alın ve istedikleri yere bırakın.
- güvenli sürün, yani çarpışma yok.
- onları mümkün olan en kısa sürede sürün.

OpenAI Gym'den [Taxi-v3](https://gym.openai.com/envs/Taxi-v3/) ortam adı verilen bir ortamı kullanacağız.

<img src="https://zephyrnet.com/wp-content/uploads/2022/01/hands-on-reinforcement-learning-course-part-2.png" width=400px/>


Bu dünya R(ed), G(reen), Y(ellow) ve B(lue) ile gösterilen dört belirlenmiş konum vardır.

Bölüm başladığında, taksi rastgele bir karede hareket eder ve yolcu rastgele bir konumdadır (R, G, Y veya B).

Taksi, yolcunun bulunduğu yere gider, yolcuyu alır, yolcunun gideceği yere (belirtilen dört konumdan başka birine) gider ve ardından yolcuyu bırakır. Bunu yaparken, taksi şoförümüzün | olarak işaretlenmiş herhangi bir duvara çarpmamak için dikkatli bir şekilde sürmesi gerekmektedir. 
Yolcu bırakıldıktan sonra bölüm sona erer.


# 2. Environment (Çevre), actions (eylemler), states(durumlar), rewards(ödüller)

👉Bir Reinforcement Learning (pekiştirmeli öğrenme) problemini çözmeden önce, ne olduğunu tanımlamanız gerekir.

- eylemler (the actions)
- dünyanın durumları (the states of the world)
- ödüller (the rewards)

👉OpenAI's gym'deki Taxi-v3 ortamını kullanıyoruz: https://gym.openai.com/envs/Taxi-v3/

👉Taxi-v3 kolay bir ortamdır çünkü eylem alanı küçüktür ve durum alanı büyüktür ancak sonludur.

👉Sınırlı sayıda eylem ve duruma sahip ortamlara tablo denir.

In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline
%config InlineBackend.figure_format = 'svg'

## Load the environment (çevreyi yükleme) 🌎

Önce ortamı yükleyelim:

In [ ]:
import gym
env = gym.make("Taxi-v3").env

Her adımda ajanın seçebileceği eylemler nelerdir?

- 0 sürücü aşağı
- 1 sürücü
- 2 sürücü sağa
- 3 sürücü sola
- 4 yolcu almak
- 5 yolcu bırakmak


## Action space (Eylem uzayı)

In [ ]:
print("Action Space {}".format(env.action_space))

Ve durumlar (states)?

- 25 olası taksi pozisyonu, çünkü dünya bir 5×5 ızgarası.
- Yolcunun R, G, Y, B artı yolcunun takside olduğu durum olmak üzere 5 olası konum
- 4 hedef konum

bu bize 25 x 5 x 4 = 500 durum verir.

## State space (Durum uzayı)

In [ ]:
print("State Space {}".format(env.observation_space))

Ödüller hakkında ne dersin?

- -1, varsayılan adım başına ödül.
Neden -1 ve sadece 0 değil? Çünkü her ekstra adımı cezalandırarak ajanı en kısa süreyi geçirmeye teşvik etmek istiyoruz. Bir taksi şoföründen beklediğiniz bu değil mi?
    
- +20 puan, yolcuyu doğru yere teslim etmek için verilen ödül.

- -10 puan, yanlış yerde alma veya bırakma işlemi gerçekleştirmenin ödülü.

Ödüller ve ortam geçişlerini (environment transitions) 
(state, action) → next_state 
`env.P` değerinden okuyabilirsiniz .

## Rewards (Ödüller)

In [ ]:
# env.P çift dictionary tipindedir.
# - 1. anahtar, 0'dan 499'a kadar olan durumu (state) temsil eder
# - 2. anahtar, ajan (agent) tarafından gerçekleştirilen eylemi (action) temsil eder,
# 0'dan 5'e

# example
state = 123
action = 0  # move south (güneye hareket)

# env.P[state][action][0] 4 elemanlı bir listedir
# (probability, next_state, reward, done)
# (olasılık, sonraki_durum, ödül, tamamlandı)
# 
#  - probability (olasılık)
#    Bu ortamda (environment) her zaman 1'dir, yani
#    belirleyen hiçbir dış/rastgele faktör yoktur.
#    next_state
#    ajanın eylemi dışında a.
#
#  - next_state: bu durumda 223
# 
#  - reward: bu durumda -1
#
#  - done: boolean (True/False) 
#          bölüm sona erdi (yani sürücü yolcu doğru varış noktasında)
print('env.P[state][action][0]: ', env.P[state][action][0])

Bu arada, bu `env.P` vektörlerinin anlamlı olup olmadığını iki kez kontrol etmek için ortamı her durum altında oluşturabilirsiniz:

`state=123`

In [ ]:
# Need to call reset() at least once before render() will work
# render() çalışması için en az bir kez reset() öğesini çağırmanız gerekiyor
env.reset()

env.s = 123
env.render(mode='human')

ajan, state=223'e ulaşmak için action=0 güneye hareket eder.

In [ ]:
env.s = 223
env.render(mode='human')